# Plagiarism Detector

<div class="alert alert-block alert-info">
<b>Note:</b> All of my string answers are written in the blue box.
</div>

# Q.1

## 1
The algorithm is as follows:

○ Store all length-k substrings of X into a hash table Tx with the hash values computed by using rolling hashing.

○ For each substring sy in Y, compute the hash value h(sy) using rolling hashing, and use this hash value to look up sy in table Tx. If the lookup is successful, then we have a common substring.

In [1]:
"""
The following algorithm is based on the model code provided in Nguyen and Tran (2021) 
accessed from https://drive.google.com/file/d/1lwkrjRKEm1v8hwqHDYRASqoIKuYJh9iP/view.

I make some modification to suit my own implementation.
"""

class HashTableNode:
    """ 
    A class that stores the information about the substring node stored in the hash table.

    Attributes
    ----------
    key: int
         an integer that is either converted from the string using ascii 
         or hashed by a hash function
    index: arr
           an array of substring's first letter's indexes in an original input string 
           that shares the same string
    string: str
            the letters of substring
    """  
    
    def __init__(self, key, index, string):
        """
        Initializes the information about the node stored in the hash table.
        
        Parameters
        ----------
        key
        index
        string
        """
        self.key = key
        self.index = [index]
        self.string = string
        
        
class HashTable:
    """ 
    A class that implements a hashtable that stores a node containing information 
    about the substring in a spot at an index computed by hash function.

    Attributes
    ----------
    m: int
        A size of hash table
    hash_table: arr
                A hash table with length m
    """  
    
    def __init__(self, m):
        """
        Initializes the information about the hash table.
        
        Parameters
        ----------
        m
        hash_table
        """
        self.m = m 
        self.hash_table = [None for _ in range(m)]
    
    def hash_function_1(self, key):
        """
        Defines a hash function which uses a division method and returns a hashed value.
        
        Parameters
        ----------
        key

        Returns
        ----------
        int
          a hashed value (-- key mod table size)
        """
        return key % self.m
    
    def hash_function_2(self, key):
        """
        Defines a second hash function which uses a division method and returns 
        a hashed value.
        
        Parameters
        ----------
        key

        Returns
        ----------
        int
          a hashed value (-- 1 + key mod (table size-1))
        """
        return 1 + key % (self.m-1)
    
    def probing_sequence(self, key, i):
        """
        Returns an index to be probed in the hash table to find an empty spot and 
        the node with potentially the same substring
        
        Parameters
        ----------
        key
        i: int
           indicates the ith time to probe a spot in the hash table
           this number is considered in the calculation
           (NOTE: when i=0, the calculation would be (the hash function 1 % tablesize))

        Returns
        ----------
        int
          an index to be probed in the hash table
        """
        #since our i keeps increasing by 1
        #we keep searching the next slot until we find an empty slot
        return (self.hash_function_1(key) + i*self.hash_function_2(key)) % self.m

    def insert_string(self, key, index, string):
        """
        Probes every slot in the hash table and
        if it finds an empty slot, inserts a node with the substing's key, index, and string,
        or if it the input string is found to be already stored in the hash table, 
        appends index to that existing node.
        
        Note: I use the open addressing as a collision resulution strategy.
        
        Parameters
        ----------
        key
        index
        string

        Returns
        ----------
        None
        """
        i = 0
        # we go through all slots
        while i < self.m:
            # find the index for storing the value
            j = self.probing_sequence(key, i)
            # if we find an empty slot
            if self.hash_table[j] is None:
                # we insert the substring's key, starting index, and string
                # into the hash table at the index derived using the hash functions
                self.hash_table[j] = HashTableNode(key, index, string) 
                break
            # or if the key is exactly the same (the sequence of strings are the same)
            elif self.hash_table[j].string == string:
                # add the index to show there are duplicates 
                self.hash_table[j].index.append(index)
                break
            # else we increase i and continue
            else:
                i += 1    
        # raise exception if all slots are probed
        if i == self.m:
            raise Exception("All slots in hash table is filled!")
            
    def match_string(self, key, string):
        """
        Matches the input string with the strings stored in the hash table by using 
        the double hashing in probing, and returns a starting index list of the matched
        substring's node.
        
        Parameters
        ----------
        key
        string

        Returns
        ----------
        boolean
        - True if the algorithm finds the matched substring's node through checking 
          a substring stored in a hash table
        
        None / Index
        - None or an index list of the matched substring's node
        """
        i = 0
        # we go through all slots
        while i < self.m:
            # find the index for storing the value
            j = self.probing_sequence(key, i)
            # if we find an empty slot, return false
            if self.hash_table[j] is None:
                return False, None
            # or we find the mathched string, return the index list
            elif self.hash_table[j].string == string:
                return True, self.hash_table[j].index
            # increase i and continue to probe
            i += 1
        return False, None

In [2]:
import math

def prime(number):
    """
    Returns a maximum prime smaller than the input integer, 
    using the Sieve of Eratosthenes algorithm.
    
    Modified the code from https://www.geeksforgeeks.org/sieve-of-eratosthenes/
    
    input
    --------
    number: int
            a integer we want to obtain a prime smaller than it.
        
    output
    -------
    integer: a maximum prime smaller than the input integer
    """
    # initialize the list as all True (all numbers are prime numbers)
    sieve = [True] * number
    # zero and one are not prime numbers
    sieve[0], sieve[1] = False, False 
    
    # create the sieve using the Sieve of Eratosthenes algorithm
    for i in range (2, int(math.sqrt(number)) + 1):
        pointer = i*2
        while pointer < number:
            sieve[pointer] = False
            pointer += i
            
    # return the prime closest to the number
    i = number - 1 # decline the index by 1 because the max 
    while sieve[i] == False:
        i -= 1
    if sieve[i] == True:
        return i

In [3]:
def rh_get_match(x, y, k):
    """
    Finds all common length-k substrings of x and y using rolling hashing on both strings.
    
    input
    --------
    x, y: strings (all white spaces are removed)
    k: int, length of substring
    
    output:
    A list of tuples (i, j) where x[i:i+k] = y[j:j+k]
    """
    ## your code here
    d = 131 # this base is the smallest prime number larger than all ascii key 
    mod = prime(int(len(x)*1.3) + 5) # hash table size should be a prime
    result = []
    hashtable = HashTable(mod) # construct a hash table

    xkey = 0
    # compute hashvalue for the first window of text
    for i in range(k):
        xkey += ord(x[i]) * d**(k-i-1) 
    # insert the key-value pair into hashtable
    hashtable.insert_string(xkey, 0, x[:k])
    # use rolling hashing to compute hashvalue for each substring
    for s in range(len(x)-k): 
        xkey = (xkey*d + ord(x[s+k])) % mod # add letter s+m
        xkey = (xkey+mod) % mod # make sure that t >= 0
        xkey = (xkey-ord(x[s]) * ((d**k) % mod)) % mod # remove letter s
        # insert the key-value pair into hashtable
        hashtable.insert_string(xkey, s+1, x[s+1:s+1+k])
    
    ykey = 0
    # compute hashvalue for the first window of text
    for j in range(k):
        ykey += ord(y[j]) * d**(k-j-1) 
    check, index = hashtable.match_string(ykey, y[:k])
    if check:
        for i in range(len(index)):
            result.append((index[i], 0))
    # use rolling hashing to compute hashvalue for each substring
    for h in range(len(y)-k): 
        ykey = (ykey*d + ord(y[h+k])) % mod # add letter s+m
        ykey = (ykey+mod) % mod # make sure that t >= 0
        ykey = (ykey-ord(y[h]) * ((d**k) % mod)) % mod # remove letter s
        # check the match of string
        check, index = hashtable.match_string(ykey, y[h+1:h+1+k])
        if check:
            for i in range(len(index)):
                result.append((index[i], h+1))
                
    return result

○ You will need to specify q for the hash function (a mod q) in rolling hashing. However, do not worry about fine-tuning q — so long as you choose a prime number and provide a justification for why your chosen value of q makes sense with regards to the table size.

○ Other design choices should be thoroughly justified, including but not limited to: the hash table’s size, data structures used, and any additional Python functions.

<div class="alert alert-block alert-info">
<b>Justification about implementation:</b> Hashing is an efficient data structure to store and search substrings and to detect plagiarism. It can save time in probing the matches by using hash functions and employing rolling hashing and double hashing. Although hashing is not suitable for finding a value given a particular input range, plagiarism detection involves more of a particular value match, so it does not concern our case.

Given that our primary goal in building hashing is to avoid hash collisions, I carefully think about the choice of base for converting the string into an integer. Here, I use the prime number to fulfill that aim. Here, I use 131 as a base because it is a prime larger than the largest ASCII code, 127 (source: http://cactus.io/tutorials/web/what-is-an-ascii-code), so that the base would not equal to any ASCII codes and the overlapping integer key can be avoided to the best extent, while the computation would not take a long time compared to the larger base value. 

It is also worth mentioning the efficiency of the collision resolution strategy since there is always a possibility of a hash collision happening. I employ double hashing, one of the open-addressing methods since it limits every slot to store only one node and takes less time to probe the node of interest by using the hash functions instead of partially using the inefficient list probing alongside the hash function in the chaining method. In implementing double hashing, I make sure that the second hash function can help search all slots in the hash table in combination with the first one. Thus, I modify the first hash function to make the second one so that they can complement each other.

In addition, I carefully think about the case where the exact same substrings are detected in one input string. Although I initially plan to store them as different nodes in distinct slots in the hash table, I realize that way can involve more probing computation (both when inserting node and finding node) if there are many repetitions in the input string. For better time efficiency, I decide to store all duplicates into one node by modifying the node’s attributes and making it store the list of an index of a substring with the exact same letters, and the letters of the substring as well.  

Last but not least, for the hash table size, I choose a prime that is approximately 1.3 times larger than the string’s length minus the length of substrings for matching (source: https://cseweb.ucsd.edu/~kube/cls/100/Lectures/lec16/lec16-8.html). This rule of thumb is useful because it considers 1) the possibility of overflowing key-value pairs by having completely filled hashtable and 2) a large required space due to the overestimated hash table size. Besides, we can transfer the hash table size as the hash function’s divisor (*q* in ‘a mod q’). Since the remainder is always smaller than the divisor, it is a perfect fit for the index in the hash table as we don’t have to re-run the hush function to find the index smaller than the hash table size. 
</div>

## 2. Test Cases
Demonstrate that your code works as expected by testing it with at least three, non-trivial test cases

<div class="alert alert-block alert-info">
I construct a function to match the strings automatically.
</div>

In [4]:
def string_matcher(x,y,k):
    """
    Finds all common length-k substrings of x and y
    without using hashing.
    
    Input:
    - x, y: str (all white spaces are removed)
    - k: int
         length of substring
    
    Output:
    - a list of tuples (i, j) where y[i:i+k] = x[j:j+k]
    """
    result = []
    
    # cross search and match all possible substrings of x and y
    for i in range(len(y)-k+1):
        for j in range(len(x)-k+1):
            if y[i:i+k] == x[j:j+k]:
                result.append((j,i))
    return result

<div class="alert alert-block alert-info">
I also build a function to convert input sentences for the suitable format for the plagiarism detector.
</div>

In [5]:
def convert_sentence(sentence):
    """
    Converts the input sentence into an all-lower-case and space-free string.
    
    Input
    - sentence: str
                a sentence we want to remove all spaces and make it lower case
        
    Output
    - strings: an all-lower-case and space-free string
    """
    lowered = sentence.lower()
    strings = ''.join(lowered.split())
    return strings

<div class="alert alert-block alert-info">
Below are the three test cases.
</div>

In [6]:
# Test case 1
"""
When sentences are free from plagiarism -- completely different two sentences --,
the algorithm should return a blank list.

Cited from https://www.collinsdictionary.com/dictionary/english/for-example
"""

s_x1 = """The man climbed up the hill"""
s_y1 = """
A driving test, for example, is something we expect all competent 
adults to be able to pass.
"""

x1 = convert_sentence(s_x1)
y1 = convert_sentence(s_y1)
assert rh_get_match(x1, y1, 3) == string_matcher(x1, y1, 3)

In [7]:
# Test case 2

"""
Typical plagiarism example with many common phrases founr in both sentences (e.g., 
values: that you work hard for what you want in life; that your word is your bond 
and you do what you say). Thie test case is also very long, so it can show how fast 
my implemented algorithm can yield the result.

Cited from https://www.usatoday.com/story/news/politics/onpolitics/2016/07/19/
melania-trump-republican-convention-speech-plagiarism/87278088/
"""


s_x2 = '''And Barack and I were raised with so many of the same values: that you work hard
for what you want in life; that your word is your bond and you do what you say 
you're going to do; that you treat people with dignity and respect, even if you 
don't know them, and even if you don't agree with them.
And Barack and I set out to build lives guided by these values, and to pass 
them on to the next generation. Because we want our children — and all children 
in this nation — to know that the only limit to the height of your achievements 
is the reach of your dreams and your willingness to work for them.
'''
s_y2 = '''From a young age, my parents impressed on me the values that you work hard for 
what you want in life, that your word is your bond and you do what you say and 
keep your promise, that you treat people with respect.
They taught and showed me values and morals in their daily lives. That is a 
lesson that I continue to pass along to our son. And we need to pass those 
lessons on to the many generations to follow. Because we want our children in 
this nation to know that the only limit to your achievements is the strength of 
your dreams and your willingness to work for them.
'''

x2 = convert_sentence(s_x2)
y2 = convert_sentence(s_y2)
assert rh_get_match(x2, y2, 3) == string_matcher(x2, y2, 3)

In [9]:
# Test case 3
"""
The worst case example where the string is built of many sets of three-letter-long 
substrings (i.e., 'ate' in this example) and another string which the algorithm would check 
against the first one is the perfect case of plagiarism -- it is a faithful copy of the first
one. Since, in my implementation, I store the exact same substrings in one slot 
(HashTableNode) by adding index to that, this test case can check if that is implemented 
correctly.
"""

s_x3 = '''ateateateateateateateateateateateateateateateateateateateateateateate
ateateateateateateateateateateateateateateateateateateateateateateateateate
'''

s_y3 = s_x3

assert rh_get_match(s_x3, s_y3, 3) == string_matcher(s_x3, s_y3, 3)

# Q.2

**instruction**

You will now work on the second version of this algorithm:

○ Store all length-k substrings of X into a hash table Tx, using a hash function that does not entail the division method.

○ For each substring sy in Y, compute the hash value h(sy) and use this hash value to look up sy in table Tx. If the lookup is successful, then we have a common substring.
Note that using a hash function that does not entail the division method prohibits the use of rolling hashing.

Asymptotically speaking, we are essentially taking more time for computing hash values in this version.

However, we are now free to choose a more fancy function, so there is a tradeoff here.

## 1. 
Provide a string reflection on what makes a good hash function. Run an experiment to give empirical evidence to support your choice for the hash function.

<div class="alert alert-block alert-info">
<b>Answer:</b> Good hash functions can reduce the possibility of a collision and produce a consistent value for the same key input.  

First, a hash function is expected to produce various hashed values to prevent a collision. Since collision requires another hashing and takes more time, the overall hashing operation can go against the initial aim of hashing, efficient storing and matching, due to the inefficient hash function. This characteristic also relates to the ability to probe all the indexes in the hashtable. A hash function needs to produce all indexes ranging from 0 to len(hashtable)-1 so that it can make sure the value can be stored in an empty spot or the value of interest can be found. This feature is especially of importance when employing double hashing since the second function is meant to complement the first one and the combination of them is significantly crucial. 

Also, the optimal hash function generates an equivalent index if it receives the same key inputs. This feature is important to find duplicates from the same string input or match from another string input because if the hash function yields a random value, it would take time to find the already-stored value that has the same key.

</div>

In [10]:
# empirical evidence

# Here are three functions I defined
def hash_function_1_mid(key):
    """
    Defines a hash function which uses a mid-square method and returns a hashed value.
    (referfence: https://research.cs.vt.edu/AVresearch/hashing/midsquare.php)

    Parameters
    ----------
    key

    Returns
    ----------
    int
      a hashed value
    """
    double = str(key**2) # make the doubled value string to use len() method in next line
    s = math.floor(len(double)/2)
    return int(double[s-1:s+1]) # rerutn integer to be computed in probing_sequence method

def hash_function_2_mid(key):
    """
    Defines a second hash function which uses a mid-square method and returns 
    a hashed value.
    (referfence: https://research.cs.vt.edu/AVresearch/hashing/midsquare.php)

    Parameters
    ----------
    key

    Returns
    ----------
    int
      a hashed value
    """
    return hash_function_1_mid(key)+1

def probing_sequence(key, i, tablesize):
    """
    Returns an index to be probed in the hash table to find an empty spot and 
    the node with potentially the same substring.

    Parameters
    ----------
    key
    i: int
       indicates the ith time to probe a spot in the hash table
       this number is considered in the calculation
       (NOTE: when i=0, the calculation would be (the hash function_1_mid % tablesize))

    Returns
    ----------
    int
      an index to be probed in the hash table
    """
    #since our i keeps increasing by 1
    #we keep searching the next slot until we find an empty slot
    return (hash_function_1_mid(key) + i*hash_function_2_mid(key)) % tablesize

In [11]:
#proof for 1st point
import random

# prepare a list of table size (prime number)
tslst = list(set(prime(i) for i in range(100,500)))

for tablesize in tslst:
    trial = 0
    
    # run trial for 30 times
    for trial in range(30):
        hash_table = [False for _ in range(tablesize)]
        key = random.randrange(1000000,2500000) # make it plausible (base: 131, use ASCII)
        
        i = 0
        while i < tablesize:
            j = probing_sequence(key, i, tablesize)
            # if the spot has not been probed
            if hash_table[j] == False:
                # change it into True
                hash_table[j] = True 
            # continue to probe other spots
            i += 1 

        #count the number of unprobed spot
        count = 0
        for k in range(tablesize):
            if hash_table[k] == False:
                count += 1  
        trial += count
    
    # take the average of trials
    trial /= 30
    # take the percentage of unprobed spots
    trial /= tablesize
    percent = round(trial*100,2)
    print(f'{percent}% of unprobed spots in a hash table with the tablesize of {tablesize}')

0.38% of unprobed spots in a hash table with the tablesize of 257
0.74% of unprobed spots in a hash table with the tablesize of 131
0.25% of unprobed spots in a hash table with the tablesize of 389
0.37% of unprobed spots in a hash table with the tablesize of 263
0.25% of unprobed spots in a hash table with the tablesize of 383
0.71% of unprobed spots in a hash table with the tablesize of 137
0.7% of unprobed spots in a hash table with the tablesize of 139
0.36% of unprobed spots in a hash table with the tablesize of 269
0.24% of unprobed spots in a hash table with the tablesize of 397
0.36% of unprobed spots in a hash table with the tablesize of 271
0.24% of unprobed spots in a hash table with the tablesize of 401
0.65% of unprobed spots in a hash table with the tablesize of 149
0.35% of unprobed spots in a hash table with the tablesize of 277
0.64% of unprobed spots in a hash table with the tablesize of 151
0.34% of unprobed spots in a hash table with the tablesize of 281
0.24% of un

<div class="alert alert-block alert-info">
<b>Answer:</b> As shown above, there is almost no unprobed spots with this hash function. Those resulting percentage below 0 is an error percentage that is negligiable because there would be multiple key inputs that complements this error.
</div>

In [12]:
#proof for the 2nd point

for i in range(10):
    for tablesize in tslst:
        for trial in range(30): # run trial for 30 times
            # store random value to represent key
            keylst1 = [random.randrange(1000000,2500000) for k in range(100)]
            # copy the list
            keylst2 = keylst1.copy()
            
            # check if the probing sequence can yield the same hashed value 
            # with the same key input
            assert(probing_sequence(keylst1[trial], i, tablesize) 
                   == probing_sequence(keylst2[trial], i, tablesize))

<div class="alert alert-block alert-info">
<b>Answer:</b> As shown above, the hash function is stable and generates consistent hashed value with the same key input.
</div>

## 2.
Implement the second version of the algorithm described above using the template below. 

In [13]:
class HashTable2(HashTable): 
    """
    A class that adds non-rolling hashing method to the parent HashTable class which
    implements a hashtable that stores a node containing information about the substring
    in a spot at an index computed by hash function.

    Attributes
    ----------
    m: int
        A size of hash table
    hash_table: arr
                A hash table with length m
    """  
        
    def __init__(self, m):
        """
        Initializes the information about the hash table.
        
        Parameters
        ----------
        m
        """
        super().__init__(m)
        
    def hash_function_1_mid(self, key):
        """
        Defines a hash function which uses a mid-square method and returns a hashed value.
        (referfence: https://research.cs.vt.edu/AVresearch/hashing/midsquare.php)
        
        Parameters
        ----------
        key

        Returns
        ----------
        int
          a hashed value
        """
        double = str(key**2) # make the doubled value string to use len() method in next line
        s = math.floor(len(double)/2)
        return int(double[s-1:s+1]) # rerutn integer to be computed in probing_sequence method
    
    def hash_function_2_mid(self, key):
        """
        Defines a second hash function which uses a mid-square method and returns 
        a hashed value.
        (referfence: https://research.cs.vt.edu/AVresearch/hashing/midsquare.php)
        
        Parameters
        ----------
        key

        Returns
        ----------
        int
          a hashed value
        """
        return self.hash_function_1_mid(key)+1
    
    def probing_sequence(self, key, i):
        """
        Returns an index to be probed in the hash table to find an empty spot and 
        the node with potentially the same substring.
        
        Parameters
        ----------
        key
        i: int
           indicates the ith time to probe a spot in the hash table
           this number is considered in the calculation
           (NOTE: when i=0, the calculation would be (the hash function_1_mid % tablesize))

        Returns
        ----------
        int
          an index to be probed in the hash table
        """
        #since our i keeps increasing by 1
        #we keep searching the next slot until we find an empty slot
        return (self.hash_function_1_mid(key) + i*self.hash_function_2_mid(key)) % self.m

In [14]:
def regular_get_match(x, y, k):
    """
    Finds all common length-k substrings of x and y
    NOT using rolling hashing on both strings.
    Input:
    - x, y: strings
    - k: int, length of substring
    Output:
    - A list of tuples (i, j)
    where x[i:i+k] = y[j:j+k]
    """
    ## your code here
    d = 131 # this base is the smallest prime number larger than all ascii key 
    tablesize = prime(int(len(x)*1.3) + 5) # hash table size should be a prime
    result = []   
    hashtable = HashTable2(tablesize) # construct a hash table

    # compute hashvalue for each substring
    for s in range(len(x)-k+1): 
        xkey = 0
        for i in range(k):
            xkey += ord(x[s+i]) * d**(k-i-1) 
        # insert the key-value pair into hashtable
        hashtable.insert_string(xkey, s, x[s:s+k])

    # compute hashvalue for each substring
    for s in range(len(y)-k+1): 
        ykey = 0
        for i in range(k):
            ykey += ord(y[s+i]) * d**(k-i-1) 
        # check the match of string
        check, index = hashtable.match_string(ykey, y[s:s+k])
        if check:
            for i in range(len(index)):
                result.append((index[i], s))   
   
    return result

### i.
Justify any design choices (the hash function, the hash table’s size, the data structures used,
and any additionally built Python functions).

<div class="alert alert-block alert-info">
<b>Justification about implementation:</b> I use the same base and hash table size as the Q1 version because the Q2 version is also interested in reducing the number of collisions and the base and hash table size defined in the Q1 version are already reasonable. 
    
The Q2 version has two features different from the Q1 one.
    
First, it inherits the HashTable class from Q1 because there are many overlapping methods we can use for Q2 as well. One thing to note is that I make the HashTable2 class overrides the probing_sequence method from the original HashTable class so that we can use match_string and insert_string methods directly (these two methods use the probing_sequence method within the process). In this way, it saves space complexity and is very efficient. 
    
Second, the regular_get_match function computes the key for every substring from scratch without using rolling hashing. Since it requires more computation, this operation takes more time than the Q1 version.

</div>

### ii.
Write clean and well-structured code, making use of PEP 8 coding conventions. Because this
version only differs from the one in question 1 in how the hash values are computed, try to
organize your code in a clear way that reutilizes as much functionality as the one you have
already provided code for (for example, consider adding the hashing method either as a
positional argument or as a method of choice if you wrap your code in a Python class).

No external libraries should be used, except for the math, random, and numpy modules.

<div class="alert alert-block alert-info">
Done above
</div>

## 3.
Illustrate how your code works by giving an explicit example. 

<div class="alert alert-block alert-info">
<b>Answer:</b> Assume that the algorithm wants to detect plagiarism between two strings, “Seoulisembeddedwithauniquesoul” (the sentence version is “Seoul is embedded with a unique soul”) and “AuniquesoulfillsSeoul” (the sentence version is “A unique soul fills Seoul”.), by using length 3 substrings. First, the algorithm inputs base for generating the integer key from substring, computes the table size by using the prime function and generating a prime around the length of the 1st string, creating a blank list to store the output, and constructing a hash table.
    
Next, upon the first string, it computes the hash value for each substring with length 3 and inserts the hash table node with relevant information (key-value, starting index in the input string, the letters of substring) into hashtable at an index of the computed hash value.
Then, upon the second string, it does the same operation except for insertion; it instead looks for the matching substrings stored in the hash table. If there is any matches, it appends the tuple of the starting index of the substring in the first and second strings into the list. 
Therefore, the output would be [(19, 0), (20, 1), (21, 2), (22, 3), (23, 4), (24, 5), (25, 6), (26, 7), (2, 8), (27, 8), (0, 16), (1, 17), (2, 18), (27, 18)].
</div>

<div class="alert alert-block alert-info">
Below are the three test cases.
I use the same test cases with the ones form Q1 to compare its required computation time.
</div>

In [15]:
# Test case 1

s_x1 = """The man climbed up the hill"""
s_y1 = """
A driving test, for example, is something we expect all competent 
adults to be able to pass.
"""

x1 = convert_sentence(s_x1)
y1 = convert_sentence(s_y1)
assert regular_get_match(x1, y1, 3) == string_matcher(x1, y1, 3)

In [16]:
# Test case 2

s_x2 = '''And Barack and I were raised with so many of the same values: that you work hard
for what you want in life; that your word is your bond and you do what you say 
you're going to do; that you treat people with dignity and respect, even if you 
don't know them, and even if you don't agree with them.
And Barack and I set out to build lives guided by these values, and to pass 
them on to the next generation. Because we want our children — and all children 
in this nation — to know that the only limit to the height of your achievements 
is the reach of your dreams and your willingness to work for them.
'''
s_y2 = '''From a young age, my parents impressed on me the values that you work hard for 
what you want in life, that your word is your bond and you do what you say and 
keep your promise, that you treat people with respect.
They taught and showed me values and morals in their daily lives. That is a 
lesson that I continue to pass along to our son. And we need to pass those 
lessons on to the many generations to follow. Because we want our children in 
this nation to know that the only limit to your achievements is the strength of 
your dreams and your willingness to work for them.
'''

x2 = convert_sentence(s_x2)
y2 = convert_sentence(s_y2)
assert regular_get_match(x2, y2, 3) == string_matcher(x2, y2, 3)

In [17]:
# Test case 3

s_x3 = '''ateateateateateateateateateateateateateateateateateateateateateateate
ateateateateateateateateateateateateateateateateateateateateateateateateate
'''

s_y3 = s_x3

assert regular_get_match(s_x3, s_y3, 3) == string_matcher(s_x3, s_y3, 3)

# Q.3

Carefully describe how you would use the code above to investigate the extent of plagiarism. Enumerate potential pitfalls and challenges of applying this algorithm for real-life use (you can also compare both of
these algorithms with a more brute-force approach to plagiarism detection; if you do this, make sure to
describe how such approach would work and why it wouldn’t be very appealing from a computational
standpoint). Make sure you justify all the assumptions you make.

<div class="alert alert-block alert-info">
<b>Answer:</b> To check the degree of plagiarism, I would use my algorithms’ outputs (the common substrings’ the starting index in both input strings), compute the relative distance between the starting indexes for each input string, and compare that value. If those two values are similar, I would assume that there is a high degree of plagiarism.
For instance, if an output is [(19, 0), (20, 1), (21, 2), (22, 3), (23, 4), (24, 5), (25, 6), (26, 7), (2, 8), (27, 8), (0, 16), (1, 17), (2, 18), (27, 18)], I would use ([19, 20] and [0, 1]) from the first two tuples of the list and compare the distance. Since this relative distance will tell you the degree of using similar words or words’ order, it is very helpful to investigate in this way.

Besides, depending on the different substring lengths we probe using the algorithms, we may gain different insights on the degree of plagiarism. Since each word has a distinct length, it does not make sense to equate the letter length of all words and compare the non-word substrings. Instead, the plagiarism detector may be made to detect the common word instead of k-length substrings that may not be a word and compare the number of shared words. That potential algorithm can be also extended to probe the phrases to have higher accuracy in plagiarism detection.
</div>

# Q.4

Discuss the time complexity of each algorithmic version. Compare and contrast the two versions by
experimenting on self-generated inputs. You are encouraged to generate inputs of various natures as that would be more likely to tease out the strengths and/or weaknesses of the versions.


<div class="alert alert-block alert-info">
<b>Answer:</b> Since I implement the search and insert methods using the open-addressing, its time complexity is at most O(1) respectively. It is because the probe function can find the ith slot occupied with a probability around $α^i (= n/m = load factor)$. From a geometric progression, the sum of those probabilities is $1/(1-α)$. Given that $α < 1$, $1/(1-α) < 1$, and therefore, it is guaranteed to have O(1) runtime. 

Assume that the input strings’ lengths are a and b respectively.

Given that the search and insert methods respectively take O(1) and rolling hashing takes O(k) time complexity when k is the length of string, the total time complexity for the Q1 version is
$O(1)*a + O(1)*b + O(a) + O(b)= O(n)$ by substituting $a+b$ with n.

The Q2 version (non-rolling hashing version) also has the same time complexity O(n) because computing the hash value for each substring from scratch requires O(1) as well and it takes O(1) * (a+b) for generating the hash value of both input strings.

Although both of them share the same asymptotic time complexity notation, it requires various time duration to detect plagiarism for different cases. The differences between Q1 and Q2 are 1) hash function and 2) the way strings are converted into integer keys. Since both versions of hash functions involve almost the same math computation which requires asymptotically equivalent time, we will focus on difference 2) mainly.

I experiment with the three versions below.
The first version is when two strings do not have any common substrings. Here, the non-rolling-hashing version (Q2) is faster than the rolling hashing version (Q1) ($(Q1, Q2) ≒ (0.0042,  0.0015)$). The Q2 version is more efficient probably because the hash function in that version effectively probes the non-matching strings. 

The second version is when two strings have many common substrings and are the case of plagiarism. Here, the rolling hashing version (Q1) is a lot faster than the non-rolling-hashing version (Q2) ($(Q1, Q2) ≒ (0.003,  0.02)$). The Q1 version is much more efficient potentially because the hash function in that version effectively probes the empty spot and store the key-value pairs with less collision. Also, the process of rolling hashing takes less time than computing every single common substring from scratch. Therefore, those two factors contribute to the saved runtime of the Q1 version.

The last version is when two strings have exactly the same strings and the strings are made of repeated substrings with the length of k. Here, the rolling hashing version (Q1) and the non-rolling-hashing version (Q2) are equivalently fast ($(Q1, Q2) ≒ (0.002,  0.002)$). It means that when the algorithm computes the integer key, there is no difference between the Q1 and Q2 versions. It may be because the strings consist of repeating substrings and that removing and adding a letter to three-letter strings is almost the same computation as calculating the integer key from three letters. 

</div>

<div class="alert alert-block alert-info">
Below are the experiments.
</div>

In [18]:
import time
k = 3

s_x1 = """The man climbed up the hill"""
s_y1 = """
A driving test, for example, is something we expect all competent 
adults to be able to pass.
"""

a1 = convert_sentence(s_x1)
b1 = convert_sentence(s_y1)

# Q1 version
count1 = 0
for i in range(30):
    start = time.time()
    rh_get_match(a1, b1, k)
    end = time.time()   
    t1_1 = end - start
    count1 += t1_1
count1 /= 30

# Q2 version
count2 = 0
for i in range(30):
    start = time.time()
    regular_get_match(a1, b1, k)
    end = time.time()  
    t1_2 = end - start
    count2 += t1_2
count2 /= 30

count1, count2

(0.0004986047744750977, 0.001496569315592448)

In [19]:
s_x2 = '''And Barack and I were raised with so many of the same values: that you work hard
for what you want in life; that your word is your bond and you do what you say 
you're going to do; that you treat people with dignity and respect, even if you 
don't know them, and even if you don't agree with them.
And Barack and I set out to build lives guided by these values, and to pass 
them on to the next generation. Because we want our children — and all children 
in this nation — to know that the only limit to the height of your achievements 
is the reach of your dreams and your willingness to work for them.
'''
s_y2 = '''From a young age, my parents impressed on me the values that you work hard for 
what you want in life, that your word is your bond and you do what you say and 
keep your promise, that you treat people with respect.
They taught and showed me values and morals in their daily lives. That is a 
lesson that I continue to pass along to our son. And we need to pass those 
lessons on to the many generations to follow. Because we want our children in 
this nation to know that the only limit to your achievements is the strength of 
your dreams and your willingness to work for them.
'''

a2 = convert_sentence(s_x2)
b2 = convert_sentence(s_y2)

# Q1 version
count1 = 0
for i in range(30):
    start = time.time()
    rh_get_match(a2, b2, k)
    end = time.time()   
    t2_1 = end - start
    count1 += t2_1
count1 /= 30

# Q2 version
count2 = 0
for i in range(30):
    start = time.time()
    regular_get_match(a2, b2, k)
    end = time.time()  
    t2_2 = end - start
    count2 += t2_2
count2 /= 30

count1, count2

(0.004288522402445475, 0.025202322006225585)

In [20]:
a3 = '''ateateateateateateateateateateateateateateateateateateateateateateate
ateateateateateateateateateateateateateateateateateateateateateateateateate
'''
b3 = a3

# Q1 version
count1 = 0
for i in range(30):
    start = time.time()
    rh_get_match(a3, b3, k)
    end = time.time()   
    t3_1 = end - start
    count1 += t3_1
count1 /= 30

# Q2 version
count2 = 0
for i in range(30):
    start = time.time()
    regular_get_match(a3, b3, k)
    end = time.time()  
    t3_2 = end - start
    count2 += t3_2
count2 /= 30

count1, count2

(0.0031581560770670572, 0.0022886037826538087)